# Script to get and analyze the APC consumption report

###### 2021-11-29
###### This script has been built to enable users to further analyze the full APC consumption report (https://TEAM.CLUSTER.celonis.cloud/integration/ui/data-consumption). This can serve as a complement to the EMS screen as this allows export and data pool grouping.

#### Inputs
None if run from the MLWB.
Token if run out of the MLW.

#### Outputs
Consumption report with used GB per table and data pool:
* as pandas DataFrame
* as CSV file

#### Processing steps
1. Import and connect
2. Fetch data
3. Process date and data volume
4. Analyze (group by Data Pool)
5. Export as CSV

## Import and connect

In [ ]:
import pandas as pd
from pycelonis import get_celonis
from datetime import datetime as dt
import time

In [ ]:
c = get_celonis()

## Fetch data

In [ ]:
def get_consumption_df(c):
    page = 0
    df=pd.DataFrame()
    
    # Iterate over pages of data consumption
    while True: # while true + if -> break 
        url = f"{c.url}/integration//api/pools/data-consumption/?limit=5000&page={page}&sort=consumptionInBytesZA"
        consumption_table = c.api_request(url, message = 'None', method = 'GET', get_json = True)
        t_list = consumption_table["extendedTableConsumptionTransports"]
        if len(t_list) == 0:
            # Reached last page: no more data
            break
        df = pd.concat([df,pd.DataFrame(t_list)])
        page += 1
        # Limit api request rate
        time.sleep(1)
    return df

df_consumption_ = get_consumption_df(c)

## Transform

In [ ]:
df_consumption = df_consumption_.copy()
df_consumption["rawDataSizeGB"] = df_consumption["rawDataSize"] / (1024**3)
df_consumption["lastUpdateDt"] = pd.to_datetime(df_consumption["lastUpdate"], unit='ms')
df_consumption = df_consumption[["dataPoolId", "dataPoolName", "tableName", "rawDataSizeGB", "lastUpdateDt"]]
df_consumption.head()    

## Analyse

In [ ]:
df_consumption_summary = df_consumption.groupby(["dataPoolId", "dataPoolName"]).agg({"rawDataSizeGB":sum, "lastUpdateDt":min}).reset_index()
df_consumption_summary = df_consumption_summary.sort_values("rawDataSizeGB", ascending=False)

In [ ]:
df_consumption_summary["rawDataSizeGB"].sum()

In [ ]:
df_consumption_summary

In [ ]:
# Details per table
df_consumption.sort_values("rawDataSizeGB", ascending=False)

## Export

In [ ]:
def to_csv(df, name):
    df.to_csv(f"{name}_{dt.now().strftime('%Y-%m-%d_%Hh%M')}.csv", sep=';', decimal=',')

In [ ]:
to_csv(df_consumption_summary, "consumption_summary")
to_csv(df_consumption, "consumption_details")